In [1]:
from numpy import random,arange
from time import time
import numpy as np
# from quicksort import do_quicksort, do_r_quicksort
from misc import generate_random, generate_figures, init_gauges_collection, append_gauges
import matplotlib.pyplot as plt

In [2]:
calls = 0

#
# Quicksort
#

def do_quicksort(A):
    global calls
    calls = 0
    quicksort(A, 0, len(A)-1)
    return calls

def quicksort(A, lo, hi):
    global calls
    calls += 1
    if lo < hi:
        p = partition(A, lo, hi)
        quicksort(A, lo, p - 1)
        quicksort(A, p + 1, hi)

def partition(A, lo, hi):
    pivot = A[hi]
    i = lo - 1
    for j in range(lo, hi):
        if A[j] < pivot:
            i = i + 1
            temp = A[j]
            A[j] = A[i]
            A[i] = temp
    if A[hi] < A[i+1]:
        temp = A[i+1]
        A[i+1] = A[hi]
        A[hi] = temp
    return i + 1

#
# Randomized QuickSort
#


def do_r_quicksort(A):
    global calls
    calls = 0
    r_quicksort(A, 0, len(A) - 1)
    return calls

def r_partition(A,lo,hi):
    rand_idx = np.random.randint(hi-lo+1)+lo
    temp = A[hi]
    A[hi] = A[rand_idx]
    A[rand_idx] = temp
    pivot = A[hi]
    i = lo - 1
    for j in range(lo, hi):
        if A[j] < pivot:
            i = i + 1
            temp = A[j]
            A[j] = A[i]
            A[i] = temp
    if A[hi] < A[i+1]:
        temp = A[i+1]
        A[i+1] = A[hi]
        A[hi] = temp
    return i + 1

def r_quicksort(A,lo,hi):
    global calls
    calls += 1
    if lo<hi:
        p = r_partition(A,lo,hi)
        r_quicksort(A, lo, p - 1)
        r_quicksort(A, p + 1, hi)



In [3]:
def measure_qs(data, sorting_function):
    '''
    Measure the execution time of sorting function [sorting_function]
    and return the number of calls for partition sorting
    '''
    t_start = time()
    calls = sorting_function(data)
    t_end = time()
    e_time = t_end - t_start
    return e_time,calls

def gauge_functions(data_size, unsortedness, repeats = 100):
    '''
    You need to measure the average execution time of a sorting function.
    The function measure_qs performs the evaluation of run time performance.
    Usage: measure_qs( data_to_sort, sorting_function)
              sorting_function: do_quicksort or do_r_quicksort
           returns: execution_time, number_of_sorting_function_calls
    '''

    # Gauges for deterministic algorithm
    det_e_time_avg=0.; det_calls_avg=0.;
    # Gauges for randomized algorithm
    rnd_e_time_avg=0.; rnd_calls_avg=0.

    for i in range(repeats):
        # For fair comparison you should feed the same data to different
        # sorting algorithms
        data = generate_random(data_size,unsortedness)

        # %%% START YOUR CODE HERE %%%
        det_e_time, det_calls = measure_qs(data, do_quicksort)
        det_e_time_avg += det_e_time
        det_calls_avg += det_calls
        
        rnd_e_time, rnd_calls = measure_qs(data, do_r_quicksort)
        rnd_e_time_avg += rnd_e_time
        rnd_calls_avg += rnd_calls
    # %%% END YOUR CODE HERE %%%
    return det_calls_avg/repeats, rnd_calls_avg/repeats, det_e_time_avg/repeats, rnd_e_time_avg/repeats

In [4]:
# Fix the seed to obtain comparable results
random.seed(0)
init_state = random.get_state()

In [5]:
##########
# Test the correctness of the quicksort algorithm
test_data = random.randint(0,10,size=100)

print("\nOriginal",test_data)

data_1 = test_data.copy()
do_quicksort(data_1)
print("Deterministic sort",data_1)

data_2 = test_data.copy()
do_r_quicksort(data_2)
print("Randomized sort   ",data_2,"\n")
print("Verification",sum(data_2-data_1),"\n")

# random.set_state(init_state)



##########
# On the first stage evaluate the impact of the dataset size on the sorting
# performance
gauges_collection = init_gauges_collection()

# Array of possible dataset sizes
data_size_rng = arange(100, 1000, 100)
x_points = len(data_size_rng)

# Fixed level of data unsortedness
unsortedness = 0.3

for ind,data_size in enumerate(data_size_rng):
    gauges = gauge_functions(data_size, unsortedness)
    append_gauges(gauges_collection, gauges)
    print("\rTesting against dataset size %d/%d" % (ind+1, x_points), end="")

generate_figures(gauges_collection, xlabel = "Input Size",xrange = data_size_rng)
##########


print("")


##########
# Evaluate the impact of the unsortedness level on the sorting performance
gauges_collection = init_gauges_collection()

# Array of possible unsortedness levels
unsortedness_level = arange(.1, 1.1, .1)
x_points = len(unsortedness_level)

# Fixed level of dataset size
data_size = 1000

for ind,unsortedness in enumerate(unsortedness_level):
    gauges = gauge_functions(data_size, unsortedness)
    append_gauges(gauges_collection, gauges)
    print("\rTesting against unsortedness level %d/%d" % (ind+1, x_points), end="")

generate_figures(gauges_collection, xlabel = "Unsortedness", xrange = unsortedness_level)
##########


print("\nFinished\n")


Original [5 0 3 3 7 9 3 5 2 4 7 6 8 8 1 6 7 7 8 1 5 9 8 9 4 3 0 3 5 0 2 3 8 1 3 3 3
 7 0 1 9 9 0 4 7 3 2 7 2 0 0 4 5 5 6 8 4 1 4 9 8 1 1 7 9 9 3 6 7 2 0 3 5 9
 4 4 6 4 4 3 4 4 8 4 3 7 5 5 0 1 5 9 3 0 5 0 1 2 4 2]
Deterministic sort [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 7 7
 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9]
Randomized sort    [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 7 7
 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9] 

Verification 0 

Testing against dataset size 9/9
Testing against unsortedness level 10/10
Finished

